In [1]:
%matplotlib inline
from __future__ import print_function, division, absolute_import
import numpy as np
from astropy import constants as const
from astropy.coordinates import Angle
from astropy import units
import matplotlib.pyplot as plt

In [2]:
# define constants
frequency = (250 * 1e6 * units.Hz)
wavelength = const.c / frequency.to(1/units.s)
earth_rot_speed = (Angle(360, units.deg) / units.sday).to(units.rad/units.s)
corr_FoV_min = Angle(10., units.degree)
hera_latitude = Angle('-30:43:17.5', units.deg)
corr_int_time = 0.1 * units.s
corr_post_fs_int_time = 10. * units.s
n_channels = 8192
corr_chan_width = (250 * units.MHz) / n_channels
max_decorr = 0.1

In [3]:
def bl_resolution(bl_len):
    # baseline angular resolution
    return Angle(np.arcsin(min(1, wavelength/(np.max(bl_len, 0.1*units.m)))), units.radian).to(units.arcminute)

def decorr_int_time(lx, ly):
    # decorrelation due to pre-fringe-stopped integration time
    x_res = bl_resolution(lx)
    y_res = bl_resolution(ly)
    dit_x = corr_int_time * earth_rot_speed / x_res.to(units.arcminute)
    dit_y = corr_int_time * earth_rot_speed * np.abs(np.sin(hera_latitude)) / y_res.to(units.arcminute)
    dit = max(dit_x, dit_y)
    return dit.value

def decorr_chan_width(bl_len):
    # decorrelation due to channel width
    dcw = (corr_chan_width.to(1/units.s) * bl_len * units.m
           * np.sin(corr_FoV_min.to(units.rad)) / const.c)
    return dcw.value

def decorr_pre_fs(lx, ly, bl_len):
    # decorrelation from pre-fringe-stopped considerations (integration time + channel width)
    dit = decorr_int_time(lx, ly)
    return 1 - (1 - dit) * (1 - decorr_chan_width(bl_len))

def dudt(lx, ly, ha):
    """
    Define partial u/ partial t from Eqn. 42 in Wijnholds et al. 2018

    Assumes that lx, ly are in units of meters.
    """
    return (lx * np.cos(ha) - ly * np.sin(ha)) * earth_rot_speed / wavelength

def dvdt(lx, ly, ha, dec):
    """
    Define partial v/ partial t from Eqn. 42 in Wijnholds et al. 2018

    Assumes that lx, ly are in units of meters.
    """
    return (lx * np.sin(dec) * np.sin(ha) + ly * np.sin(dec) * np.cos(ha)) * earth_rot_speed / wavelength

def decorr_post_fs_int_time(lx, ly, int_time):
    # find the maximum decorrelation for the source 10 degrees off zenith in +/- l and +/- m

    # case 1: +l
    du = dudt(lx, ly, corr_FoV_min)
    dv = dvdt(lx, ly, corr_FoV_min, hera_latitude)
    l = np.cos(90*units.deg + corr_FoV_min)
    m = 0.
    rfac = (du * l + dv * m)**2
    
    # case 2: -l
    du = dudt(lx, ly, -corr_FoV_min)
    dv = dvdt(lx, ly, corr_FoV_min, hera_latitude)
    l = np.cos(90*units.deg - corr_FoV_min)
    m = 0.
    rfac = max(rfac, (du * l + dv * m)**2)
    
    # case 3: +m
    du = dudt(lx, ly, 0.)
    dv = dvdt(lx, ly, 0., hera_latitude + corr_FoV_min)
    l = 0.
    m = np.cos(90*units.deg + corr_FoV_min)
    rfac = max(rfac, (du * l + dv * m)**2)
    
    # case 4: -m
    du = dudt(lx, ly, 0.)
    dv = dvdt(lx, ly, 0., hera_latitude - corr_FoV_min)
    l = 0.
    m = np.cos(90*units.deg - corr_FoV_min)
    rfac = max(rfac, (du * l + dv * m)**2)

    # add to other factors
    return np.pi**2 * int_time.value**2 / 6. * rfac.value

def decorr_total(lx, ly, bl_len, int_time):
    return 1 - (1 - decorr_pre_fs(lx, ly, bl_len)) * (1 - decorr_post_fs_int_time(lx, ly, int_time))

def fs_int_time(lx, ly, decorr_val):
    # fringe-stopped integration time for a given baseline separation and decorrelation value

    # case 1: +l
    du = dudt(lx, ly, corr_FoV_min)
    dv = dvdt(lx, ly, corr_FoV_min, hera_latitude)
    l = np.cos(90*units.deg + corr_FoV_min)
    m = 0.
    rfac = (du * l + dv * m)**2
    
    # case 2: -l
    du = dudt(lx, ly, -corr_FoV_min)
    dv = dvdt(lx, ly, corr_FoV_min, hera_latitude)
    l = np.cos(90*units.deg - corr_FoV_min)
    m = 0.
    rfac = max(rfac, (du * l + dv * m)**2)
    
    # case 3: +m
    du = dudt(lx, ly, 0.)
    dv = dvdt(lx, ly, 0., hera_latitude + corr_FoV_min)
    l = 0.
    m = np.cos(90*units.deg + corr_FoV_min)
    rfac = max(rfac, (du * l + dv * m)**2)
    
    # case 4: -m
    du = dudt(lx, ly, 0.)
    dv = dvdt(lx, ly, 0., hera_latitude - corr_FoV_min)
    l = 0.
    m = np.cos(90*units.deg - corr_FoV_min)
    rfac = max(rfac, (du * l + dv * m)**2)

    # invert to get integration time
    return np.sqrt(6 * decorr_val / (np.pi**2 * rfac.value))

def max_int_time(lx, ly, bl_len, max_decorr):
    # Compute the maximum post-fring-stopped integration time for a given baseline
    # length, in m, and max decorrelation fraction
    # Assumes fringe stopping
    dpf = decorr_pre_fs(lx, ly, bl_len)
    dfs = 1 - (1 - max_decorr)/(1 - dpf)
    int_time = fs_int_time(lx, ly, dfs)

    return int_time

In [4]:
hera_txt = '/Users/plaplant/Documents/school/penn/software/hera_mc/hera_mc/data/HERA_350.txt'
hera_bls = np.genfromtxt(hera_txt)

In [5]:
lx = 870 * units.m
ly = 0. * units.m
length = np.sqrt(lx**2 + ly**2)
dt = decorr_total(lx, ly, length, corr_post_fs_int_time)
nit = max_int_time(lx, ly, length, 0.1)
r = nit / corr_post_fs_int_time.value
fac = np.floor(np.log2(r))
print("decorr_total: ", dt)
print("new_int_time: ", nit)
print("ratio: ", r)
print("fac: ", fac)

decorr_total:  0.0286371700403
new_int_time:  25.2646734331
ratio:  2.52646734331
fac:  1.0


/Users/plaplant/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:26: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return umr_maximum(a, axis, None, out, keepdims)
/Users/plaplant/anaconda2/lib/python2.7/site-packages/astropy/units/quantity.py:1059: RuntimeWarning: divide by zero encountered in true_divide
  return super(Quantity, self).__truediv__(other)


In [6]:
# compute the savings for each baseline in the array
# XXX: takes ~5 minutes; might be able to be more clever, but easier to shut up and calculate
max_bda_comp_factor = 0.
simple_bda_comp_factor = 0.

nants = hera_bls.shape[0]
for iant in range(nants):
    # first column is antenna name; second--fourth columns are xyz positions, in meters
    xi = hera_bls[iant, 1]
    yi = hera_bls[iant, 2]
    for jant in range(iant + 1, nants):
        xj = hera_bls[jant, 1]
        yj = hera_bls[jant, 2]

        # assume EW separation is Delta(x) and NS separation is Delta(y)--true-ish to 1st order
        lx = np.abs(xi - xj) * units.m
        ly = np.abs(yi - yj) * units.m
        
        # get total length
        length = np.sqrt(lx**2 + ly**2)
        
        # compute max decorrelation value along principal axes
        dt = decorr_total(lx, ly, length, corr_post_fs_int_time)

        if dt < max_decorr:
            # we can theoretically integrate this bl in time until we hit the max_decorr
            new_int_time = max_int_time(lx, ly, length, max_decorr)
            max_bda_comp_factor += corr_post_fs_int_time.value / new_int_time

            # also compute the max power-of-two integration factor
            fac = np.floor(np.log2(new_int_time / corr_post_fs_int_time.value))
            simple_bda_comp_factor += 2.**(-fac)
            
            # drive home the point that the default correlator output rates are overkill for most baselines
            if fac == 0:
                print("No compression savings possible for this baseline")
        else:
            # no savings
            max_bda_comp_factor += 1
            simple_bda_comp_factor += 1

# add factor for autos; assume no compression
# note that it doesn't really matter what we put, since these are < 1% of baselines
max_bda_comp_factor += nants
simple_bda_comp_factor += nants

# normalize by the number of baselines
nbls = (nants * (nants + 1)) / 2
max_bda_comp_factor /= nbls
simple_bda_comp_factor /= nbls

In [7]:
print("Theoretic maximum baseline-dependent averaging compression factor for HERA-350 array: ")
print(max_bda_comp_factor, "\n")
print("Power-of-2 compression factor: ")
print(simple_bda_comp_factor)

Theoretic maximum baseline-dependent averaging compression factor for HERA-350 array: 
0.0641460074895 

Power-of-2 compression factor: 
0.0897276912902


In [8]:
# compute data rate for season
channels_to_keep = n_channels * 3. / 4.
sum_diff_factor = 2
bytes_per_vis = 8 * units.byte
n_polarizations = 4
obs_hrs_per_day = 12 * units.hour / units.day
days_per_season = 120 * units.day

naive_data_rate = (channels_to_keep * nbls * n_polarizations * bytes_per_vis
                   * sum_diff_factor / corr_post_fs_int_time)
naive_data_vol = naive_data_rate * obs_hrs_per_day * days_per_season
bda_data_rate = simple_bda_comp_factor * naive_data_rate
bda_data_vol = bda_data_rate * obs_hrs_per_day * days_per_season
print("Naive data rate:      ", naive_data_rate.to(units.Gbyte/units.s))
print("Naive season volume:  ", naive_data_vol.to(units.Pbyte), "\n")
print("BDA data rate:        ", bda_data_rate.to(units.Gbyte/units.s))
print("BDA season volume:    ", bda_data_vol.to(units.Pbyte))

Naive data rate:       2.41532928 Gbyte / s
Naive season volume:   12.5210669875 Pbyte 

BDA data rate:         0.21672192 Gbyte / s
BDA season volume:     1.12348643328 Pbyte
